In [ ]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

from transformers import TFAutoModelForSequenceClassification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [ ]:
df_init = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/wagon/books_processed_top60.csv', engine="python")
df_init

,Author,Book
0,Twain Mark,﻿The Project Gutenberg EBook of Chapters from ...
1,Ebers Georg,"﻿The Project Gutenberg EBook Margery, by Georg..."
2,Motley John Lothrop,﻿The Project Gutenberg EBook The Life of John ...
3,Warner Charles Dudley,"﻿The Project Gutenberg EBook of England, by Ch..."
4,Ebers Georg,﻿The Project Gutenberg EBook In The Fire Of Th...
...,...,...
959,Warner Charles Dudley,"﻿The Project Gutenberg EBook of As We Go, by C..."
960,Ebers Georg,﻿The Project Gutenberg EBook A Word Only A Wor...
961,Montaigne Michel de,﻿The Project Gutenberg Etext of Quotations fro...
962,Motley John Lothrop,﻿The Project Gutenberg EBook of The Life of Jo...


In [ ]:
def strip_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
def clean_special_chars(text):
    clean = re.sub(r'[\n\r\t]', '', text)
    return clean
df_init['Book'] = df_init['Book'].apply(strip_html)
df_init['Book'] = df_init['Book'].apply(clean_special_chars)

def chunkator(text, LEN=300):
    txt = text.split()
    chunks = [txt[i:i+LEN] for i in range(0, len(txt), LEN)]
    # chunked_text = ' '.join([' '.join(chunk) for chunk in chunks])
    return chunks

# Create an empty list to store the chunks
chunks_list = []

# Iterate over each row in the original DataFrame
for index, row in df_init.iterrows():
    author = row['Author']
    book = row['Book']

    # Split the book text into chunks of 512 characters
    chunks = chunkator(book)

    # Create a DataFrame with author and book chunks
    chunk_df = pd.DataFrame({'Author': author, 'Book': chunks})

    # Append the chunk DataFrame to the list
    chunks_list.append(chunk_df)

# Concatenate all the chunk DataFrames into a new DataFrame
new_df = pd.concat(chunks_list, ignore_index=True)

# Print the new DataFrame
new_df

,Author,Book
0,Twain Mark,"[﻿The, Project, Gutenberg, EBook, of, Chapters..."
1,Twain Mark,"[are, of, asort, which, he, is, familiar, with..."
2,Twain Mark,"[at, any, time, during, its, existence, who, w..."
3,Twain Mark,"[and, unchangingly, bitter, against, Charles,a..."
4,Twain Mark,"[same, difficulty., In, fact, he, looked, dist..."
...,...,...
292655,Shelley Percy Bysshe,"[refund, of, any, money, paid, by, a, user, wh..."
292656,Shelley Percy Bysshe,"[work, under, this, agreement,, disclaim, alll..."
292657,Shelley Percy Bysshe,"[invalidity, or, unenforceability, of, anyprov..."
292658,Shelley Percy Bysshe,"[U.S., federal, laws, and, your, state's, laws..."


In [ ]:
for index, row in new_df.iterrows():
    # Convert the list of words into a string
    book_string = ' '.join(row['Book'])

    # Update the "Book" column with the string value
    new_df.at[index, 'Book'] = book_string

new_df

,Author,Book
0,Twain Mark,﻿The Project Gutenberg EBook of Chapters from ...
1,Twain Mark,are of asort which he is familiar with in his ...
2,Twain Mark,at any time during its existence who would be ...
3,Twain Mark,"and unchangingly bitter against Charles,and I ..."
4,Twain Mark,"same difficulty. In fact he looked distraught,..."
...,...,...
292655,Shelley Percy Bysshe,refund of any money paid by a user who notifie...
292656,Shelley Percy Bysshe,"work under this agreement, disclaim allliabili..."
292657,Shelley Percy Bysshe,invalidity or unenforceability of anyprovision...
292658,Shelley Percy Bysshe,U.S. federal laws and your state's laws.The Fo...


In [ ]:
top40 = new_df['Author'].value_counts().head(40).index.tolist()

In [ ]:
new_df = new_df[new_df['Author'].isin(top40)]

In [ ]:
amount_per_author = new_df['Author'].value_counts().min()

In [ ]:
new_df = new_df.groupby('Author').head(amount_per_author)

In [ ]:
names_to_drop = ['Benton Thomas Hart', 'Spenser Edmund', 'Browning Robert', 'Warner Charles Dudley', 'Mill John Stuart']

# Drop rows where the names appear in the "Author" column
new_df = new_df[~new_df['Author'].isin(names_to_drop)]

In [ ]:
# get the labels
labels = new_df['Author'].tolist()

# convert labels to nparray
labels_text = np.array(labels)

# apply label encoding to labels
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(labels_text)
labels = np.array(labels)

labels_clean = le.inverse_transform(labels)

new_df['Author_Encoded']=labels

In [ ]:
new_df

,Author,Book,Author_Encoded
0,Twain Mark,﻿The Project Gutenberg EBook of Chapters from ...,29
1,Twain Mark,are of asort which he is familiar with in his ...,29
2,Twain Mark,at any time during its existence who would be ...,29
3,Twain Mark,"and unchangingly bitter against Charles,and I ...",29
4,Twain Mark,"same difficulty. In fact he looked distraught,...",29
...,...,...,...
273713,Poe Edgar Allan,lacerating the feet against the pavement was l...,18
273714,Poe Edgar Allan,"all the turrets around about Jerusalem, and th...",18
273715,Poe Edgar Allan,and hast sojourned among them who dabble with ...,18
273716,Poe Edgar Allan,rope became indistinctly visible—“Booshoh he!”...,18


In [ ]:
shuffled_df = new_df.sample(frac = 1)

In [ ]:
shuffled_df_val_test = shuffled_df.tail(18696).copy()
df_test = shuffled_df_val_test.tail(9348).copy()
df_val = shuffled_df_val_test.head(9348).copy()
df_train = shuffled_df.copy().drop(shuffled_df.tail(18696).index)

In [ ]:
list_text_train = []
for _, row in df_train.iterrows():
    list_text_train.append(row["Book"])

list_labels_train =[]
for _, row in df_train.iterrows():
    list_labels_train.append(row["Author_Encoded"])

list_text_val = []
for _, row in df_val.iterrows():
    list_text_val.append(row["Book"])

list_labels_val =[]
for _, row in df_val.iterrows():
    list_labels_val.append(row["Author_Encoded"])

list_text_test = []
for _, row in df_test.iterrows():
    list_text_test.append(row["Book"])

list_labels_test =[]
for _, row in df_test.iterrows():
    list_labels_test.append(row["Author_Encoded"])

In [ ]:
data_dict_train = {'text': list_text_train, 'labels': list_labels_train}
data_dict_val = {'text': list_text_val, 'labels': list_labels_val}
data_dict_test = {'text': list_text_test, 'labels': list_labels_test}

In [ ]:
from datasets import Dataset

ds_train = Dataset.from_dict(data_dict_train)
ds_val = Dataset.from_dict(data_dict_val)
ds_test = Dataset.from_dict(data_dict_test)

In [ ]:
from datasets import DatasetDict

ds_final = DatasetDict({
    'train': ds_train,
    'test': ds_test,
    'val': ds_val})

ds_final

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 63099
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 9348
    })
    val: Dataset({
        features: ['text', 'labels'],
        num_rows: 9348
    })
})

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_ds_final = ds_final.map(preprocess_function, batched=True)

Map:   0%|          | 0/63099 [00:00<?, ? examples/s]

Map:   0%|          | 0/9348 [00:00<?, ? examples/s]

Map:   0%|          | 0/9348 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
data_tmp = df_train[['Author', 'Author_Encoded']].groupby(['Author_Encoded']).min()

id2label = data_tmp.to_dict()['Author']

label2id = {v: k for k, v in id2label.items()}

id2label, label2id

({0: 'Auerbach Berthold',
  1: 'Brown Goold',
  2: 'Dumas Alexandre',
  3: 'Ebers Georg',
  4: 'Fishburne William Brett',
  5: 'Grant Ulysses S. (Ulysses Simpson)',
  6: 'Haeckel Ernst',
  7: 'Hardy Thomas',
  8: 'Holmes Oliver Wendell',
  9: 'Josephus Flavius',
  10: 'Larned J. N. (Josephus Nelson)',
  11: 'Lemprière John',
  12: 'Lewis Meriwether',
  13: 'Lincoln Abraham',
  14: 'Montaigne Michel de',
  15: 'Motley John Lothrop',
  16: 'Nuttall P. Austin',
  17: 'Plutarch',
  18: 'Poe Edgar Allan',
  19: 'Polo Marco',
  20: 'Rose J. Holland (John Holland)',
  21: 'Rose Joshua',
  22: 'Rymer James Malcolm',
  23: 'Shakespeare William',
  24: 'Shelley Percy Bysshe',
  25: 'Smith Adam',
  26: 'Steele Richard Sir',
  27: 'Thomas Aquinas Saint',
  28: 'Tolstoy Leo graf',
  29: 'Twain Mark',
  30: 'Walsh James J. (James Joseph)',
  31: 'Whewell William',
  32: 'Whittier John Greenleaf',
  33: 'Wood J. G. (John George)',
  34: 'Zola Émile'},
 {'Auerbach Berthold': 0,
  'Brown Goold': 1,
  '

In [ ]:
import tensorflow as tf

In [ ]:
from transformers import create_optimizer

batch_size = 32
num_epochs = 20
batches_per_epoch = len(tokenized_ds_final["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=1e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=35, id2label=id2label, label2id=label2id)

# Set the classifier layer to be trainable
model.classifier.trainable = True
# Make the BERT layers non-trainable
for layer in model.layers:
    if "bert" in layer.name:
        layer.trainable = False

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_ds_final["train"],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
tokenized_ds_final["train"]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 74784
})

In [ ]:
tf_test_set = model.prepare_tf_dataset(
    tokenized_ds_final["test"],
    shuffle=False,
    batch_size=32,
    collate_fn=data_collator,
)

In [ ]:
tf_validation_set = model.prepare_tf_dataset(
    tokenized_ds_final["val"],
    shuffle=False,
    batch_size=32,
    collate_fn=data_collator,
)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  26915     
                                                                 
 dropout_39 (Dropout)        multiple                  0         
                                                                 
Total params: 66,980,387
Trainable params: 617,507
Non-trainable params: 66,362,880
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(
    monitor="val_accuracy",
    patience=6,
    restore_best_weights=True,
    verbose=1)

from keras.callbacks import ModelCheckpoint

checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/wagon/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              save_weights_only=True,
                              verbose=1)

from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
                              patience=3, min_lr=0.0000001)

In [ ]:
history = model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=20, callbacks=[es, cp_callback, reduce_lr], verbose=1)

Epoch 1/20
1971/1971 [==============================] - ETA: 0s - loss: 4.7701 - accuracy: 0.1449
Epoch 1: saving model to /content/drive/MyDrive/Colab Notebooks/wagon/checkpoints/cp.ckpt
1971/1971 [==============================] - 191s 93ms/step - loss: 4.7701 - accuracy: 0.1449 - val_loss: 3.3285 - val_accuracy: 0.2778 - lr: 9.5003e-06
Epoch 2/20
1971/1971 [==============================] - ETA: 0s - loss: 3.6606 - accuracy: 0.3400
Epoch 2: saving model to /content/drive/MyDrive/Colab Notebooks/wagon/checkpoints/cp.ckpt
1971/1971 [==============================] - 181s 92ms/step - loss: 3.6606 - accuracy: 0.3400 - val_loss: 2.7456 - val_accuracy: 0.6308 - lr: 9.0003e-06
Epoch 3/20
1971/1971 [==============================] - ETA: 0s - loss: 3.3418 - accuracy: 0.4437
Epoch 3: saving model to /content/drive/MyDrive/Colab Notebooks/wagon/checkpoints/cp.ckpt
1971/1971 [==============================] - 181s 92ms/step - loss: 3.3418 - accuracy: 0.4437 - val_loss: 2.6550 - val_accuracy: 0

In [ ]:
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/wagon/GutenBERT_Final/GutenBERT_Model_35', save_format="tf")
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/wagon/GutenBERT_Final/GutenBERT_Tokenizer_35')

('/content/drive/MyDrive/Colab Notebooks/wagon/GutenBERT_Final/GutenBERT_Tokenizer_35/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/wagon/GutenBERT_Final/GutenBERT_Tokenizer_35/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/wagon/GutenBERT_Final/GutenBERT_Tokenizer_35/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/wagon/GutenBERT_Final/GutenBERT_Tokenizer_35/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/wagon/GutenBERT_Final/GutenBERT_Tokenizer_35/tokenizer.json')

In [ ]:
from datasets import load_metric
accuracy = load_metric('accuracy')
precision = load_metric('precision')

f1 = load_metric('f1')
recall = load_metric('recall')

<ipython-input-67-93ded6abb0eb>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy = load_metric('accuracy')


In [ ]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=1)

In [ ]:
list_dict_pred = pipe(ds_test['text'])

Token indices sequence length is longer than the specified maximum sequence length for this model (542 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
import json

json_data = json.dumps(list_dict_pred)

with open('/content/drive/MyDrive/Colab Notebooks/wagon/list_dict_pred_35.txt', 'w') as file:
  file.write(json_data)

In [ ]:
labels_name = [item['label'] for sublist in list_dict_pred for item in sublist]

In [ ]:
labels_numerical = [label2id[label] for label in labels_name]

In [ ]:
final_accuracy = accuracy.compute(predictions=labels_numerical, references=ds_test['labels'])
final_accuracy

{'accuracy': 0.7023962344886607}

In [ ]:
final_precision = precision.compute(predictions=labels_numerical, references=ds_test['labels'], average=None)
final_precision

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'precision': array([0.40163934, 0.        , 0.73366834, 0.        , 1.        ,
        0.8641115 , 0.91735537, 0.58469945, 0.6377551 , 0.        ,
        0.74796748, 0.82911392, 0.97154472, 0.78599222, 0.52920962,
        0.87136929, 0.87777778, 0.57474227, 0.82080925, 0.4452975 ,
        0.75714286, 0.95505618, 0.84536082, 0.94699647, 0.63174603,
        0.78448276, 0.50914634, 0.88745981, 0.93006993, 0.34349593,
        0.90909091, 0.6497006 , 0.51750973, 0.88461538, 0.64820847])}

In [ ]:
precision_list = final_precision['precision'].tolist()
precision_list

[0.4016393442622951,
 0.0,
 0.7336683417085427,
 0.0,
 1.0,
 0.8641114982578397,
 0.9173553719008265,
 0.5846994535519126,
 0.6377551020408163,
 0.0,
 0.7479674796747967,
 0.8291139240506329,
 0.9715447154471545,
 0.7859922178988327,
 0.5292096219931272,
 0.8713692946058091,
 0.8777777777777778,
 0.5747422680412371,
 0.8208092485549133,
 0.44529750479846447,
 0.7571428571428571,
 0.9550561797752809,
 0.845360824742268,
 0.9469964664310954,
 0.6317460317460317,
 0.7844827586206896,
 0.5091463414634146,
 0.887459807073955,
 0.9300699300699301,
 0.3434959349593496,
 0.9090909090909091,
 0.6497005988023952,
 0.5175097276264592,
 0.8846153846153846,
 0.6482084690553745]

In [ ]:
author_precision_dict = {id2label[idx]: precision for idx, precision in enumerate(precision_list)}
author_precision_dict

{'Auerbach Berthold': 0.4016393442622951,
 'Brown Goold': 0.0,
 'Dumas Alexandre': 0.7336683417085427,
 'Ebers Georg': 0.0,
 'Fishburne William Brett': 1.0,
 'Grant Ulysses S. (Ulysses Simpson)': 0.8641114982578397,
 'Haeckel Ernst': 0.9173553719008265,
 'Hardy Thomas': 0.5846994535519126,
 'Holmes Oliver Wendell': 0.6377551020408163,
 'Josephus Flavius': 0.0,
 'Larned J. N. (Josephus Nelson)': 0.7479674796747967,
 'Lemprière John': 0.8291139240506329,
 'Lewis Meriwether': 0.9715447154471545,
 'Lincoln Abraham': 0.7859922178988327,
 'Montaigne Michel de': 0.5292096219931272,
 'Motley John Lothrop': 0.8713692946058091,
 'Nuttall P. Austin': 0.8777777777777778,
 'Plutarch': 0.5747422680412371,
 'Poe Edgar Allan': 0.8208092485549133,
 'Polo Marco': 0.44529750479846447,
 'Rose J. Holland (John Holland)': 0.7571428571428571,
 'Rose Joshua': 0.9550561797752809,
 'Rymer James Malcolm': 0.845360824742268,
 'Shakespeare William': 0.9469964664310954,
 'Shelley Percy Bysshe': 0.6317460317460317,


In [ ]:
final_recall = recall.compute(predictions=labels_numerical, references=ds_test['labels'], average=None)
final_recall

{'recall': array([0.86572438, 0.        , 0.55513308, 0.        , 0.9929078 ,
        0.96498054, 0.88446215, 0.75352113, 0.45454545, 0.        ,
        0.70498084, 0.95620438, 0.9372549 , 0.76515152, 0.5480427 ,
        0.75      , 0.89097744, 0.86100386, 0.6173913 , 0.90272374,
        0.79104478, 0.98455598, 0.6007326 , 0.94366197, 0.78346457,
        0.70817121, 0.65490196, 0.96503497, 0.47841727, 0.59929078,
        0.73825503, 0.83783784, 0.52777778, 0.76666667, 0.76833977])}

In [ ]:
recall_list = final_recall['recall'].tolist()
author_recall_dict = {id2label[idx]: recall for idx, recall in enumerate(recall_list)}
author_recall_dict

{'Auerbach Berthold': 0.8657243816254417,
 'Brown Goold': 0.0,
 'Dumas Alexandre': 0.5551330798479087,
 'Ebers Georg': 0.0,
 'Fishburne William Brett': 0.9929078014184397,
 'Grant Ulysses S. (Ulysses Simpson)': 0.9649805447470817,
 'Haeckel Ernst': 0.8844621513944223,
 'Hardy Thomas': 0.7535211267605634,
 'Holmes Oliver Wendell': 0.45454545454545453,
 'Josephus Flavius': 0.0,
 'Larned J. N. (Josephus Nelson)': 0.7049808429118773,
 'Lemprière John': 0.9562043795620438,
 'Lewis Meriwether': 0.9372549019607843,
 'Lincoln Abraham': 0.7651515151515151,
 'Montaigne Michel de': 0.5480427046263345,
 'Motley John Lothrop': 0.75,
 'Nuttall P. Austin': 0.8909774436090225,
 'Plutarch': 0.861003861003861,
 'Poe Edgar Allan': 0.6173913043478261,
 'Polo Marco': 0.9027237354085603,
 'Rose J. Holland (John Holland)': 0.7910447761194029,
 'Rose Joshua': 0.9845559845559846,
 'Rymer James Malcolm': 0.6007326007326007,
 'Shakespeare William': 0.9436619718309859,
 'Shelley Percy Bysshe': 0.7834645669291339,

In [ ]:
final_f1 = f1.compute(predictions=labels_numerical, references=ds_test['labels'], average=None)
final_f1

{'f1': array([0.54871221, 0.        , 0.63203463, 0.        , 0.99644128,
        0.91176471, 0.90060852, 0.65846154, 0.53078556, 0.        ,
        0.72583826, 0.88813559, 0.95409182, 0.77543186, 0.53846154,
        0.80614203, 0.88432836, 0.68933539, 0.70471464, 0.59640103,
        0.77372263, 0.96958175, 0.70235546, 0.94532628, 0.69947276,
        0.74437628, 0.5728988 , 0.92462312, 0.63182898, 0.43669251,
        0.81481481, 0.73187184, 0.52259332, 0.82142857, 0.70318021])}

In [ ]:
f1_list = final_f1['f1'].tolist()
author_f1_dict = {id2label[idx]: f1 for idx, f1 in enumerate(f1_list)}
author_f1_dict

{'Auerbach Berthold': 0.5487122060470324,
 'Brown Goold': 0.0,
 'Dumas Alexandre': 0.632034632034632,
 'Ebers Georg': 0.0,
 'Fishburne William Brett': 0.9964412811387899,
 'Grant Ulysses S. (Ulysses Simpson)': 0.9117647058823529,
 'Haeckel Ernst': 0.900608519269777,
 'Hardy Thomas': 0.6584615384615384,
 'Holmes Oliver Wendell': 0.5307855626326964,
 'Josephus Flavius': 0.0,
 'Larned J. N. (Josephus Nelson)': 0.7258382642998028,
 'Lemprière John': 0.8881355932203391,
 'Lewis Meriwether': 0.9540918163672656,
 'Lincoln Abraham': 0.7754318618042227,
 'Montaigne Michel de': 0.5384615384615384,
 'Motley John Lothrop': 0.8061420345489444,
 'Nuttall P. Austin': 0.8843283582089553,
 'Plutarch': 0.6893353941267387,
 'Poe Edgar Allan': 0.7047146401985113,
 'Polo Marco': 0.5964010282776349,
 'Rose J. Holland (John Holland)': 0.7737226277372262,
 'Rose Joshua': 0.9695817490494297,
 'Rymer James Malcolm': 0.702355460385439,
 'Shakespeare William': 0.945326278659612,
 'Shelley Percy Bysshe': 0.6994727

In [ ]:
# SHAKESPEARE
# text = '''"To thine own self be true,
# And it must follow, as the night the day,
# Thou canst not then be false to any man.
# Farewell, my blessing season this in thee!'''

# SHAKESPEARE GPT
# text = '''"In yonder chamber, dim and forlorn, do shadows sway like ethereal specters in a haunting ballet. The wavering candle's feeble glow doth struggle against the encroaching gloom, as though 'twere reluctant to unveil the secrets lurking therein. The air hangs heavy with an oppressive silence, broken only by the distant tolling of a mournful clock, its chime beseeching the hours. Upon a withered bough, a solitary raven perches, its obsidian plumes reflecting the pallor of the chamber. And in the corner, an aged manuscript lies abandoned, its timeworn words murmuring forgotten tales and souls lost to time's cruel grasp."'''

# EDGAR ALLAN POE
# text = '''"Yet mad I am not...and very surely do I not dream. But tomorrow I die, and to-day I would unburden my soul. My immediate purpose is to place before the world, plainly, succinctly, and without comment, a series of mere household events. In their consequences, these events have terrified — have tortured — have destroyed me. Yet I will not attempt to expound them. To me, they have presented little but Horror — to many they will seem less terrible than baroques. Hereafter, perhaps, some intellect may be found which will reduce my phantasm to the common-place — some intellect more calm, more logical, and far less excitable than my own, which will perceive, in the circumstances I detail with awe, nothing more than an ordinary succession of very natural causes and effects." - from "The Black Cat"'''

# EDGAR ALLAN GPT
# text = '''"In the dim and desolate chamber, shadows danced upon the walls like specters in a macabre ballet. The flickering candle, casting its feeble light, barely pierced the gloom, as if afraid to unveil the secrets lurking within. The air was heavy with an oppressive silence, broken only by the distant echoes of a mournful clock, tolling the hours with a melancholic chime. A solitary raven perched upon a withered branch, its ebony feathers reflecting the pallor of the room. And in the corner, a worn and tattered manuscript lay abandoned, its words whispering of forgotten tales and lost souls." - Edgar Allan Poe'''

# PLATO
# text = '''Until philosophers are kings, or the kings and princes of this world have the spirit and power of philosophy, and political greatness and wisdom meet in one, and those commoner natures who pursue either to the exclusion of the other are compelled to stand aside, cities will never have rest from their evils — no, nor the human race, as I believe — and then only will this our State have a possibility of life and behold the light of day.'''

# PLATO GPT
# text = '''"In the realm of knowledge, within the dimly lit chamber of contemplation, shadows of ideas dance upon the walls, whispering of profound truths and eternal forms. The flickering flame of reason, casting its gentle light, strives to illuminate the depths of understanding, yearning to reveal the essence of existence. The air is filled with a silent quest, where seekers of wisdom navigate the intricacies of thought. Like a philosopher's stone, the mind seeks to transmute ignorance into enlightenment. And in the midst of this metaphysical tapestry, the soul engages in a symphony of inquiry, seeking to grasp the immutable realities that lie beyond the fleeting realm of appearances."'''

# RANDOM ARTICLE
# text = '''This millennium, Apple has introduced a string of landmark products: the definitive portable music player, the most popular tablet computer, those ubiquitous wireless earbuds, an iconic lightweight laptop and the standard issue coder’s laptop, a new way to pay, a smartwatch that outsells the entire Swiss watch industry.'''

# TOLSTOY
# text = '''"All, everything that I understand, I understand only because I love. Everything is, everything exists, only because I love. Everything is united by it alone. Love is God, and to die means that I, a particle of love, shall return to the general and eternal source." - Leo Tolstoy'''

# TOLSTOY GPT
# text = '''"In the realm of existence, where the boundaries of understanding unfold, I perceive that love holds the key to comprehending all things. It is through love's lens that the world becomes illuminated, revealing the intricate connections that bind everything together. Love, the divine force that permeates the fabric of existence, breathes life into every moment, infusing meaning and purpose into the grand tapestry of life. To love is to partake in the divine essence, to merge with the eternal source from which all things emanate. In the embrace of love, mortality fades, and the individual soul reunites with the cosmic whole, dissolving into the boundless expanse of universal love."'''

# text = "Good morning, my name is Arthur and I love to code"

# text = "i mean i always get told 'damn i thought we were going to just go to the market and buy tomatoes'"

# ADAM SMITH GPT
text = "The division of labor is a fundamental principle that drives the wealth and prosperity of nations. When individuals specialize in specific tasks and exchange the surplus of their production, the overall productivity of society increases. By focusing on their unique skills and engaging in mutually beneficial trade, people can harness the power of specialization to create abundance. Through the invisible hand of the market, self-interest motivates individuals to seek their own gain, but in doing so, they unintentionally promote the common good. As goods and services flow freely in the market, competition and innovation are fostered, leading to economic growth and the improvement of living standards for all members of society."

# PLUTARCH GPT
# text = "Now the object of life is not to be on the side of the majority, but to escape finding oneself in the ranks of the insane. It is a difficult task, since the majority of men are inclined to follow the path of least resistance, yielding to the opinions and customs prevailing in their society. Yet, true virtue lies in resisting such conformity and embracing reason and moral principles. It is the mark of a noble soul to stand apart, guided by wisdom and virtue, rather than being swayed by popular opinion. In the end, the worth of a person is not measured by their outward success or recognition, but by the integrity of their character and the steadfastness of their principles"

# PLUTARCH REAL
# text ="After Sylla became master of Rome, he wished to make Caesar put away his wife Cornelia, daughter of Cinna, the late sole ruler of the commonwealth, but was unable to effect it either by promises or intimidation, and so contented himself with confiscating her dowry. The ground of Sylla's hostility to Caesar was the relationship between him and Marius; for Marius, the elder, married Julia, the sister of Caesar's father, and had by her the younger Marius, who consequently was Caesar's first cousin. And though at the beginning, while so many were to be put to death, and there was so much to do, Caesar was overlooked by Sylla, yet he would not keep quiet, but presented himself to the people as a candidate for the priesthood, though he was yet a mere boy. Sylla, without any open opposition, took measures to have him rejected, and in consultation whether he should be put to death, when it was urged by some that it was not worth his while to contrive the death of a boy, he answered, that they knew little who did not see more than one Marius in that boy."



In [ ]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=3)
pipe(text)

[[{'label': 'Plutarch', 'score': 0.03786633908748627},
  {'label': 'Lemprière John', 'score': 0.03581720218062401},
  {'label': 'Larned J. N. (Josephus Nelson)', 'score': 0.033340875059366226}]]